In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import re
import nbimporter
import sys
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix 
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu
from operator import itemgetter

mods = ['exploratory_analysis', 'prep_and_split_data']
[sys.modules.pop(mod) for mod in mods if mod in sys.modules]

from exploratory_analysis import save_obj, load_obj, returnDataOnDate
from prep_and_split_data import prepareForModel, returnXandY

In [3]:
train_and_cv = pd.read_csv('data/training_and_cv_data.csv')

In [4]:
train_and_cv = prepareForModel(train_and_cv)

In [6]:
train_XY, train_X, train_Y = returnXandY(train_and_cv, '2021-06-02', '2021-06-26')
cv_XY, cv_X, cv_Y = returnXandY(train_and_cv, '2021-06-27', '2021-06-30')

In [7]:
print(f'train shape: {train_X.shape}')
print(f'cv shape: {cv_X.shape}')

train shape: (4709, 16)
cv shape: (524, 16)


## Neural Network

In [9]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)

Here is a StackExchange answer that provides a starting point for deciding on the number of hidden units to include: https://stats.stackexchange.com/a/136542

Below is a computation to obtain the absolute maximum number of hidden units:

In [10]:
Ns = train_X_scaled.shape[0]  # training examples
No = 1                           # output neurons
Ni = train_X_scaled.shape[1]  # input neurons
alpha = 2                        # scale factor

Nh = Ns / (alpha*(Ni + No))      # maximum hidden neurons

print(f'An upper bound for number of hidden units: {int(Nh)}')

An upper bound for number of hidden units: 138


This seems like *quite* a lot.

Another rule of thumb I've found is that each hidden layer should have ~$\sqrt{Ni\cdot No}$ neurons, which results in 4 neurons per layer.

Let's start with this and a single hidden layer.

In [8]:
tf.random.set_seed(40)
nn_model = Sequential(
    [               
        tf.keras.Input(shape=(train_X_scaled.shape[1],)),
        Dense(units=4, activation='relu', name='layer1'),
        #Dense(units=2, activation='relu', name='layer2'),
        Dense(units=1, activation='linear', name='layer3')
    ], name = "nn_model" 
)

In [23]:
nn_model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
)

nn_model_fit = nn_model.fit(
    train_X_scaled, train_Y,
    epochs=100
)

Epoch 1/100
148/148 [==============================] - 1s 1ms/step - loss: 159.0682
Epoch 2/100
148/148 [==============================] - 0s 2ms/step - loss: 159.0298
Epoch 3/100
148/148 [==============================] - 0s 2ms/step - loss: 159.0325
Epoch 4/100
148/148 [==============================] - 0s 2ms/step - loss: 159.0139
Epoch 5/100
148/148 [==============================] - 0s 2ms/step - loss: 158.9985
Epoch 6/100
148/148 [==============================] - 0s 1ms/step - loss: 158.9702
Epoch 7/100
148/148 [==============================] - 0s 2ms/step - loss: 158.9958
Epoch 8/100
148/148 [==============================] - 0s 1ms/step - loss: 158.9515
Epoch 9/100
148/148 [==============================] - 0s 2ms/step - loss: 158.9697
Epoch 10/100
148/148 [==============================] - 0s 1ms/step - loss: 158.9685
Epoch 11/100
148/148 [==============================] - 0s 1ms/step - loss: 158.9569
Epoch 12/100
148/148 [==============================] - 0s 1ms/step - loss

In [10]:
train_Y_preds = nn_model.predict(train_X_scaled)
cv_X_scaled = scaler.fit_transform(cv_X)
cv_Y_preds = nn_model.predict(cv_X_scaled)

d2 = {'TrainPreds': np.reshape(train_Y_preds, (train_Y_preds.size)), 'TrainVals': train_Y}
dfTrain = pd.DataFrame(data=d2)
print(dfTrain.head(15))
print()
d1 = {'cvPreds': np.reshape(cv_Y_preds, (cv_Y_preds.size)), 'cvVals': cv_Y}
dfcv = pd.DataFrame(data=d1)
print(dfcv.head(15))

17/17 [==============================] - 0s 2ms/step
     TrainPreds  TrainVals
229    7.040806  -0.499239
230    9.257483   3.456387
231    7.835962   8.400529
232    6.803262   0.946665
233    8.651533  -0.475129
234   12.507952  12.418294
235    8.197610  17.796711
236    8.760069   0.411522
237    9.316175   0.665847
238    6.555308   1.136853
239    6.845170   0.946665
240   12.592384  -1.010101
241    6.010951   0.744565
242    6.425060   0.035832
243    6.503365   0.783289

      TestPreds   TestVals
4938  11.942869  -2.854838
4939   9.343163  -1.503758
4940  11.697374  -2.854838
4941   7.181939  -1.304511
4942  11.943060  -2.854838
4943   9.810583  57.142857
4944  13.463144   5.569006
4945  12.083153   3.240172
4946   6.387346  -0.502828
4947  12.261545  -0.645893
4948   6.589577  15.042064
4949  12.175924   9.557329
4950  12.176893   9.557329
4951   8.686260   3.240172
4952   7.135557  57.142857


In [37]:
[layer1, layer2] = nn_model.layers
W1,b1 = layer1.get_weights()
print(W1)

[[ 0.33955058  0.5153292   0.25442672 -0.49963948]
 [ 0.8090537  -0.04677701 -0.8416188  -0.43038285]
 [-0.10451187  0.25276995 -0.9639532   0.3444593 ]
 [-1.4364793   0.43277752 -1.6644655   0.6104626 ]
 [-0.08666021  0.2980548   0.57271516  0.58109915]
 [-0.29034543  0.90744245  1.0659096  -0.31378835]
 [-0.1928127   0.08368501 -1.3508987   0.15999238]
 [ 0.5388136   0.42414576 -3.051762   -0.8180944 ]
 [-0.08579298  1.2191445   2.056615   -1.0101562 ]
 [ 0.8512917  -0.2672444  -0.08676212 -0.9677913 ]
 [-1.28855    -0.23726587  0.8006748   0.36757672]
 [ 0.7694906   0.09131569 -0.7260561   0.7202867 ]
 [ 0.0503447  -1.3305098  -1.0657406   0.3260037 ]
 [-0.19754073  0.2613251   0.3447797  -1.0783085 ]
 [ 0.7727951  -0.05427537 -0.61826134 -0.3786807 ]]


In [40]:
from tensorflow.python.ops import math_ops, numpy_ops
numpy_ops.np_config.enable_numpy_behavior()

def asymmetric_loss(wgt):
    def custom_loss(y_true, y_pred):
        diff = wgt/2*math_ops.squared_difference(y_pred, y_true)*(y_true < y_pred).astype(float) + \
                1/2*math_ops.squared_difference(y_pred, y_true)*(y_true >= y_pred).astype(float)
        
        loss = tf.reduce_mean(diff, axis=-1)

        return loss
    return custom_loss

In [51]:
tf.random.set_seed(40)
nn_model_custom = Sequential(
    [               
        tf.keras.Input(shape=(train_X_scaled.shape[1],)),
        Dense(units=4, activation='relu', name='layer1'),
        #Dense(units=2, activation='relu', name='layer2'),
        Dense(units=1, activation='linear', name='layer3')
    ], name = "nn_model_custom" 
)

#weights = nn_model_custom.get_weights()
#reset_model = lambda model: model.set_weights(weights)

nn_model_custom.compile(
    loss=asymmetric_loss(10.),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
)

nn_model_fit_custom = nn_model_custom.fit(
    train_X_scaled, train_Y,
    epochs=40
)

Epoch 1/40
148/148 [==============================] - 1s 2ms/step - loss: 111.9242
Epoch 2/40
148/148 [==============================] - 0s 2ms/step - loss: 109.8485
Epoch 3/40
148/148 [==============================] - 0s 2ms/step - loss: 109.3248
Epoch 4/40
148/148 [==============================] - 0s 2ms/step - loss: 108.7265
Epoch 5/40
148/148 [==============================] - 0s 2ms/step - loss: 108.4271
Epoch 6/40
148/148 [==============================] - 0s 2ms/step - loss: 108.1274
Epoch 7/40
148/148 [==============================] - 0s 2ms/step - loss: 107.8281
Epoch 8/40
148/148 [==============================] - 0s 2ms/step - loss: 107.5461
Epoch 9/40
148/148 [==============================] - 0s 2ms/step - loss: 107.5751
Epoch 10/40
148/148 [==============================] - 0s 2ms/step - loss: 107.4102
Epoch 11/40
148/148 [==============================] - 0s 2ms/step - loss: 107.1128
Epoch 12/40
148/148 [==============================] - 0s 2ms/step - loss: 107.1392
E

In [52]:
train_Y_preds = nn_model_custom.predict(train_X_scaled)
cv_X_scaled = scaler.fit_transform(cv_X)
cv_Y_preds = nn_model_custom.predict(cv_X_scaled)

d2 = {'TrainPreds': np.reshape(train_Y_preds, (train_Y_preds.size)), 'TrainVals': train_Y}
dfTrain = pd.DataFrame(data=d2)
print(dfTrain.head(15))
print()
d1 = {'cvPreds': np.reshape(cv_Y_preds, (cv_Y_preds.size)), 'cvVals': cv_Y}
dfcv = pd.DataFrame(data=d1)
print(dfcv.head(15))

17/17 [==============================] - 0s 3ms/step
     TrainPreds  TrainVals
229    1.646459  -0.499239
230    2.330482   3.456387
231    3.286908   8.400529
232    0.876429   0.946665
233    1.923940  -0.475129
234    2.633893  12.418294
235    3.772491  17.796711
236    1.885907   0.411522
237    2.469335   0.665847
238    1.028203   1.136853
239    0.860074   0.946665
240    3.585289  -1.010101
241    0.999431   0.744565
242    1.269123   0.035832
243    1.097346   0.783289

      TestPreds   TestVals
4938   3.782362  -2.854838
4939   3.193410  -1.503758
4940   5.931743  -2.854838
4941   1.544710  -1.304511
4942   3.782386  -2.854838
4943   3.282927  57.142857
4944   6.605915   5.569006
4945   3.757955   3.240172
4946   1.073820  -0.502828
4947   3.112996  -0.645893
4948   5.879294  15.042064
4949   3.519294   9.557329
4950   3.524330   9.557329
4951   1.513879   3.240172
4952   2.501349  57.142857
